In [33]:
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.7/844.7 kB 7.7 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.168.0-py2.py3-none-any.whl size=1151035 sha256=17f48b209f8deec19d3de33cfae4b322e59e9ea877a4da421d7ee8fac73714c9
  Stored in directory: /root/.cache/pip/wheels/c6/06/4f/71f8d1bfa307ba75786b2900cd9538f7282dc4b45a576bad5f
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.167.0
    Uninstalling sagemaker-2.167.0:
      Successfully uninstalled sagemaker-2.167.0

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [34]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker import get_execution_role

role = get_execution_role()
# Hub Model configuration. <https://huggingface.co/models> 'sentence-transformers/all-mpnet-base-v2'
hub = {
  'HF_MODEL_ID':'sentence-transformers/msmarco-MiniLM-L-6-v3', # model_id from hf.co/models
  'HF_TASK':'feature-extraction' # NLP task you want to use for predictions
}

s3_location = 's3://spotify-genius-featurestore/models/lyrics-embeder/model.tar.gz'

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub, # configuration for loading model from Hub
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.26.0", # transformers version used
   pytorch_version="1.13.1", # pytorch version used
   py_version='py39' 

)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge",
)

# example request, you always need to define "inputs"
data = {
"inputs": "sample text to encode"
}

# request
predictor.predict(data)

--------!

[[[0.038555532693862915,
   -0.10791001468896866,
   -0.15906229615211487,
   -0.10295748710632324,
   0.5570353865623474,
   -0.14951494336128235,
   -0.22143225371837616,
   0.3018251359462738,
   0.4424278438091278,
   -0.18282011151313782,
   -0.5350373387336731,
   0.1932791769504547,
   0.11196861416101456,
   -0.22350342571735382,
   -0.05844375491142273,
   0.33965182304382324,
   -0.061690717935562134,
   0.005008528474718332,
   0.03640121966600418,
   -0.15187662839889526,
   0.7698947191238403,
   0.09911546111106873,
   0.5244364142417908,
   -0.32554277777671814,
   0.006242275238037109,
   0.31711363792419434,
   -0.31123167276382446,
   -0.5909760594367981,
   -0.14821411669254303,
   -0.7831739783287048,
   0.18464745581150055,
   0.1342809796333313,
   0.8712478876113892,
   0.3435309827327728,
   0.5813705921173096,
   -0.07101161032915115,
   -0.3245736062526703,
   0.1546768844127655,
   0.265639066696167,
   0.0714147686958313,
   -0.5283233523368835,
   0.0415904

In [35]:
data = {
"inputs": ["i appriciate his work", "i really like his work"]
}

def cls_pooling(model_output):
    return [sublist[0] for sublist in model_output]

# result = cls_pooling(predictor.predict(data))
result = predictor.predict(data)


# print(len(sum(result[0])))

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print(len(result[0][0][1]))
for i in range(len(result[0][0])):
    print(cosine_similarity(np.array(result[0][0][i]).reshape(1, -1), np.array(result[1][0][1]).reshape(1, -1)))






384
[[-0.03840612]]
[[0.79202433]]
[[0.16285397]]
[[0.07530879]]
[[0.11417219]]
[[0.41603114]]
[[0.25604883]]
[[-0.03720824]]
